In [ ]:
import json
import uuid

# Path to the input template file
input_template_path = 'participant_template.json'

# Path for the output file with filled UUIDs
output_file_path = 'filled_participant_template.json'

def generate_uuids_for_template(input_path, output_path):
    # Load the JSON template
    with open(input_path, 'r') as file:
        data = json.load(file)

    # Function to update all relevant fields with a new UUID
    def update_with_uuids(participant):
        new_uuid = str(uuid.uuid4())
        participant['id'] = new_uuid  # Update participant ID
        
        # Update subjectReference in all resources
        for resource in participant.get('resources', []):
            resource['subjectReference'] = new_uuid

    update_with_uuids(data)

    # Save the updated data back to a new JSON file
    with open(output_path, 'w') as file:
        json.dump(data, file, indent=2)

# Call the function with the specified paths
generate_uuids_for_template(input_template_path, output_file_path)

print(f"Updated template saved to {output_file_path}")


In [ ]:
import pandas as pd

df = pd.read_csv('26_person.csv')

# clear the rows
#df = df.iloc[0:0]

# for every column names filter out until first \ but copy the splitted part to first row
examples = df.columns.str.split('(').str[1]

# make the first row the 'examples'
# df.loc[0] = examples

df.columns = df.columns.str.split('\n').str[0]

# delete the column called 'In case you are a PhD student from RTG, what is your research project number ?'
df = df.drop('In case you are a PhD student from RTG, what is your research project number ?', axis=1)

# remove 'Timestamp' column
df = df.drop('Timestamp', axis=1)

In [ ]:
df

In [ ]:
# loop over each 'what is your name' and add uuid code instead

import pandas as pd
import uuid

# Sample dataframe creation

# Dictionary to hold name-UUID mappings
name_uuid_mapping = {}

# Loop through the column and replace each entry with a UUID
for index, row in df.iterrows():
    # Generate a unique UUID
    unique_uuid = str(uuid.uuid4())
    
    # Store the original name and its corresponding UUID
    name_uuid_mapping[row['What is your name?']] = unique_uuid
    
    # Replace the original name with the UUID in the dataframe
    df.at[index, 'What is your name?'] = unique_uuid

# The dataframe now has UUIDs instead of original names


In [ ]:
# Assuming df is your current dataframe with UUIDs in 'name_column'
# and name_uuid_mapping is the dictionary containing the original name-UUID mappings

# Loop through the dataframe
#for index, row in df.iterrows():
#    # Retrieve the original name using the UUID from the dataframe
#    original_name = [name for name, uuid in name_uuid_mapping.items() if uuid == row['What is your name?']][0]
#    
#    # Replace the UUID with the original name in the dataframe
#    df.at[index, 'What is your name?'] = original_name

# Now df will have the original names restored



In [ ]:
name_uuid_mapping

In [ ]:
# change all the 'NaN's to 'No answer'
df = df.fillna('No answer')

In [ ]:
import json
# save uuid mapping to json file
with open('name_uuid_mapping.json', 'w') as file:
    json.dump(name_uuid_mapping, file)

In [ ]:
#primpt column names


In [ ]:
df.to_csv('26_person_ready.csv', index=False)

In [ ]:
import csv
import os
import re

# Define the source CSV file and the target directory for the text files
source_csv = '26_person_ready.csv'
target_directory = './individual_texts/'

# Create the target directory if it does not exist
if not os.path.exists(target_directory):
    os.makedirs(target_directory)

# Read the CSV file and write each row to a separate text file
with open(source_csv, mode='r', encoding='utf-8') as csv_file:
    csv_reader = csv.DictReader(csv_file)  # Use DictReader to access columns by name
    
    for row in csv_reader:
        # Extract the name and create a valid filename from it
        # Remove disallowed characters and spaces, replace with underscores
        
        name = row["What is your name?"]
        filename = re.sub(r'[^\w\s-]', '', name).replace(' ', '_')

        row.pop('What is your name?')
        # Define the path for the text file
        text_file_path = os.path.join(target_directory, f'{filename}.txt')
        
        # Write the content to the text file, excluding the name column
        # and any question-answer pair where the answer is 'no answer'
        content = ' | '.join([f"{k}: {v}" for k, v in row.items() if v.lower() != 'no answer'])
        with open(text_file_path, mode='w', encoding='utf-8') as text_file:
            text_file.write(content)
